Lets examine the links and scrape 


In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests


In [31]:
all_url = "https://www.allrecipes.com/recipes-a-z-6735880"
url = "https://www.allrecipes.com/air-fryer-hearts-of-palm-sticks-recipe-7814346"

Block to scrape the actual recipe

In [127]:
start=url.rfind('-')+1
rkey=url[start:]

In [82]:
response=requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the HTML content of the page using BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    #WHERE all recipe data is stored
    soup = soup.body.main.article
else:
    print("Failed to retrieve the web page.")



Build recipe dictionary

In [143]:
recipe ={}
recipe['id_recipe']=rkey

Grab the prep block info

In [144]:
rlabels = soup.find_all("div", {"class": "mntl-recipe-details__label"})
rinfo = soup.find_all("div", {"class": "mntl-recipe-details__value"})
print(rinfo)


[<div class="mntl-recipe-details__value">15 mins</div>, <div class="mntl-recipe-details__value">12 mins</div>, <div class="mntl-recipe-details__value">20 mins</div>, <div class="mntl-recipe-details__value">47 mins</div>, <div class="mntl-recipe-details__value">2 </div>]


In [145]:
label=[lab.text for lab in rlabels]
value=[val.text for val in rinfo]

recipe.update(dict(zip(label,value)))


In [146]:
display(recipe)

{'id_recipe': '7814346',
 'Prep Time:': '15 mins',
 'Cook Time:': '12 mins',
 'Freeze Time:': '20 mins',
 'Total Time:': '47 mins',
 'Servings:': '2 '}

Get Nutritional header data

In [161]:
#grab nutritional data
# Find the <th> element with the specified class
th_element = soup.find('thead', class_='mntl-nutrition-facts-label__table-head')
soup_head = th_element
# Check if the <th> element was found
if th_element:
    # Find the <span> element within the <th> element
    span_element = soup_head.find_all('span')
    # Check if the <span> element was found
    if span_element:
        #2nd and 4th element are the data values
        recipe[span_element[0].text.strip()]=span_element[1].text.strip()
        recipe[span_element[2].text.strip()]=span_element[3].text.strip()
    else:
        #recipe['Servings Per Recipe']=''
        print("The <span> element with class 'mntl-nutrition-facts-label__table-head-pretext' was not found within the <th> element.")
else:
    print("The <th> element with class 'mntl-nutrition-facts-label__table-head-subtitle' was not found in the HTML.")


In [163]:
#grab nutritional data
# Find the <th> element with the specified class
tr_element = soup.find('tbody', class_="mntl-nutrition-facts-label__table-body type--cat")
soup_body = tr_element
# Check if the <th> element was found

Grab ingredients - this is a seperate DF


In [108]:
ul_element = soup.find('ul', class_='mntl-structured-ingredients__list')

grab ingredients

In [150]:


# Find the <ul> element with the specified class
ul_element = soup.find('ul', class_='mntl-structured-ingredients__list')

# Check if the <ul> element was found
if ul_element:
    # Find all <li> elements within the <ul> element
    li_elements = ul_element.find_all('li', class_='mntl-structured-ingredients__list-item')
    
    # Initialize lists to store extracted data
    quantities = []
    units = []
    ingredients = []
    
    # Iterate through the <li> elements and extract data
    for li in li_elements:
        span_quantity = li.find('span', {'data-ingredient-quantity': 'true'})
        span_unit = li.find('span', {'data-ingredient-unit': 'true'})
        span_ingredient = li.find('span', {'data-ingredient-name': 'true'})
        
        if span_quantity:
            quantities.append(span_quantity.text.strip())
        else:
            quantities.append('')
        
        if span_unit:
            units.append(span_unit.text.strip())
        else:
            units.append('')
        
        if span_ingredient:
            ingredients.append(span_ingredient.text.strip())
        else:
            ingredients.append('')
    
    # Print the extracted data
    temp_ing_df = pd.DataFrame({'id_recipe':rkey,'Quantity': quantities, 'Unit of Measure': units, 'Ingredient': ingredients})
    
else:
    print("The <ul> element with class 'mntl-structured-ingredients__list' was not found in the HTML.")

,id_recipe,Quantity,Unit of Measure,Ingredient
0,7814346,1/4,cup,all-purpose flour
1,7814346,1/4,teaspoon,salt
2,7814346,1/8,teaspoon,ground black pepper
3,7814346,1,large,egg
4,7814346,1/4,cup,talian bread crumbs
5,7814346,2,tablespoons,panko bread crumbs
6,7814346,2,tablespoons,grated Parmesan cheese
7,7814346,1 (14,ounce) can,"hearts of palm, drained"
8,7814346,,,cooking spray
